In [ ]:
import os
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns
from util import prepare_df

In [ ]:
df = prepare_df(xp_identifier="utilisation_statespace")

In [ ]:
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}
task_palette = sns.color_palette("deep", int(df["n_task"].unique().shape[0]))
sns.set_theme(style="whitegrid", palette="deep")

In [ ]:
s = df.shape[0] * [True]
df_plot = df.loc[s]
df_piv = df_plot.copy().pivot(
    index="tid",
    columns="search_type",
    values=["duration_s", "n_visited", "schedulable", "nbt", "U"],
)
df_piv.columns = list(map(lambda x: "_".join(x), df_piv.columns))
df_piv["n_visited_ratio"] = (
    (df_piv["n_visited_ACBFS"] / df_piv["n_visited_BFS"]) - 1
) * 100
df_piv["duration_s_ratio"] = (
    (df_piv["duration_s_ACBFS"] / df_piv["duration_s_BFS"]) - 1
) * 100

df_piv.rename(columns={"nbt_BFS": "Number of tasks"}, inplace=True)
df_piv.rename(columns={"schedulable_ACBFS": "Schedulable"}, inplace=True)
df_piv.rename(columns={"U_ACBFS": "Utilisation"}, inplace=True)

In [ ]:
def fmt_percent(x, *args):
    return ("+" if x > 0 else "") + f"{x:.0f}%"


def fmt_percent_100(x, *args):
    return f"{x*100:.0f}%"

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(12, 5.5129))

ax_reduc = axs[0]
ax_sched = axs[1]

sns.lineplot(
    data=df_piv,
    x="Utilisation",
    y="n_visited_ratio",
    errorbar=None,
    ax=ax_reduc,
    label="Number of visited states before halt",
    marker="o",
)
sns.lineplot(
    data=df_piv,
    x="Utilisation",
    y="duration_s_ratio",
    errorbar=None,
    ax=ax_reduc,
    label="Execution time",
    linestyle="dashed",
    marker="o",
)
sns.lineplot(
    data=df_piv,
    x="Utilisation",
    y="Schedulable",
    errorbar=None,
    ax=ax_sched,
    marker="o",
)

ax_reduc.yaxis.set_major_formatter(fmt_percent)
ax_sched.yaxis.set_major_formatter(fmt_percent_100)
ax_reduc.xaxis.set_major_formatter(fmt_percent_100)
ax_sched.xaxis.set_major_formatter(fmt_percent_100)

ax_reduc.set_ylabel("Relative change")
ax_sched.set_ylabel("Acceptance ratio")

ax_reduc.set_xlabel("Average utilisation")
ax_sched.set_xlabel("Average utilisation")

f.savefig(
    f"./{datetime.now().strftime('%Y%m%d_%H%M%S')}-statespace-utilisation.pdf",
    bbox_inches="tight",
)

In [ ]:
df_piv["n_visited_ratio"].median()

In [ ]:
df_piv["duration_s_ratio"].median()